### Load an image and pass it to a model

In [5]:
from skimage import io, img_as_float, img_as_ubyte
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
#import torchvision.models as models
import torch.nn.functional as F

import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
#significant input from https://amaarora.github.io/2020/09/13/unet.html and Pytorch documentation examples

In [6]:
minutes = 15
#def name_to_hrs (r): return float(round(float(os.path.basename(r)[0:-4].split("_")[1][1:])*(minutes/60)+5,2))
def name_to_hrs (r): return float(round(float(os.path.basename(r)[0:-4].split("_")[1][1:])*(minutes/60)+5,2))
time = name_to_hrs('D:/pytorch/data/2D_FishAge_pytorch/images/S000_t000028_V000_R0005_X000_Y000_C02_I0_D0_P00344_MP.tif')
device = 'cuda'
img_train_paths = glob('D:/pytorch/data/2D_FishAge_pytorch/images/*.tif')
img_test_paths = glob('D:/pytorch/data/2D_FishAge_pytorch/testimages/*.tif')


In [7]:
#time
#model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=1, out_channels=1, init_features=8, pretrained=True)

In [8]:
# Code for a standard unet block that takes in in_ch and results in out_ch
# 3x3 conv and no padding currently
class Block(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3,1,1)
        self.norm1 = nn.BatchNorm2d(out_ch, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu  =nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3,1,1)
        self.norm2 = nn.BatchNorm2d(out_ch, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    
    def forward(self, x):
        return self.relu(self.norm2(self.conv2(self.relu(self.norm1(self.conv1(x))))))


In [9]:
class Encoder(nn.Module):
    def __init__(self, chs = (1,64,128,256,512,1024)):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        features = []
        for block in self.enc_blocks:
            x=block(x)
            features.append(x)
            x=self.pool(x)
        return features 

In [10]:
class Decoder(nn.Module):
    def __init__(self, chs=(1024,512,256,128,64)):
        super().__init__()
        self.chs = chs
        self.upconvs = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2,2) for i in range(len(chs)-1)])
        self.dec_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])

    def forward(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x=self.upconvs[i](x)
            adjusted_encoder_features = self.crop(encoder_features[i], x)
            x = torch.cat([x, adjusted_encoder_features], dim=1)
            x = self.dec_blocks[i](x)
        return x
    
    def crop(self, adjusted_encoder_features, x):
        _,_,H,W = x.shape
        adjusted_encoder_features = transforms.CenterCrop([H,W])(adjusted_encoder_features)
        return adjusted_encoder_features
        

In [11]:
#num_classes is the number of channels in the output. Not really desired as we want a mask
#change output to layer with highest value?
class UNet(nn.Module):
    def __init__(self, encoding_channels= (1,64,128,256,512,1024), decoding_channels = (1024, 512, 256, 128, 64), num_class = 512, retain_dim=False):
        super().__init__()
        self.encoder = Encoder(encoding_channels)
        self.decoder = Decoder(decoding_channels)
        self.head = nn.Conv2d(decoding_channels[-1], num_class, 1)
        self.retain_dim = retain_dim
    def forward(self, x):
        encoding_features = self.encoder(x)
        #Note [::-1] flips the order of the encoding channels so they start with 1024
        out = self.decoder(encoding_features[::-1][0], encoding_features[::-1][1:])
        out = self.head(out)
        if self.retain_dim:
            out = F.interpolate(out, (512,512))
        return out

In [12]:
# figure out how to use one layer of pretrained weights https://github.com/avijit9/forces/blob/master/model.py


In [13]:
nn.Flatten()

Flatten(start_dim=1, end_dim=-1)

In [14]:

class regression_net(nn.Module):
    def __init__(self, y_range = (4.9, 24)):
        super(regression_net, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.flat = nn.Flatten()
        self.norm1 = nn.BatchNorm1d(512, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True)
        self.drop1 = nn.Dropout(p=0.25, inplace=False)
        self.linear1 = nn.Linear(in_features = 512, out_features = 256, bias = False)
        self.relu = nn.ReLU()
        self.norm2 = nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.drop2 = nn.Dropout(p=0.25, inplace=False)
        self.fc = nn.Linear(256, 1, bias=False)
        self.y_range = y_range
    def forward(self, x):
        x = self.fc(self.drop2(self.norm2(self.relu(self.linear1(self.drop1(self.norm1(self.flat(self.pool(x)))))))))
        #print(x.shape)
        #Final age should be between 5 and 24 hours
   

        return (torch.sigmoid(x)*(self.y_range[1]-self.y_range[0])+self.y_range[0])
# self.pool = nn.AdaptiveAvgPool2d((32,32))
model = UNet()
regnet = regression_net()
model = nn.Sequential(model, regnet)
##########
model.to(device)

Sequential(
  (0): UNet(
    (encoder): Encoder(
      (enc_blocks): ModuleList(
        (0): Block(
          (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Block(
          (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU()
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): Block(
          (conv1): Conv2d(128, 256, kernel_size=(

In [15]:
# model.to(device)
# input = torch.from_numpy( img )[None, None, :].float()
# input = input.to('cuda')
# output = model(input)
# output.shape

In [16]:
model.train()
#def loss_fn (output, target): return 0.5*(output- target)**2
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2) # 0.0001

### use dataloader

In [17]:
pair_transforms = transforms.Compose([
    
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Resize((512,512)),
    transforms.Normalize(mean = 0.5, std=0.5),
    #transforms.ColorJitter(contrast = 0.2),
    transforms.RandomVerticalFlip(0.3),
    transforms.RandomHorizontalFlip(0.3),
    transforms.RandomApply([transforms.RandomRotation((90,90))], p=0.5)

])

In [18]:
class ImageValuePair(Dataset):
    def __init__(self, img_paths, transforms=None):
        self.img_paths = img_paths
        
        if transforms is not None:
            self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = img_as_float(io.imread(self.img_paths[idx]))
        age = torch.tensor(name_to_hrs(self.img_paths[idx]))
        img = self.transforms(img)
        return img, age

In [19]:
device = 'cuda'
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        y = y.view(-1,1)
        #print(X[200:250, 200:250])
        # Compute prediction error
        pred = model(X)

        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(pred)
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            N = len(X)
            #print(batchLen)
            X, y = X.to(device), y.to(device)
            pred = model(X) # N x 1 x H x W
            test_loss += loss_fn(pred, y).item()
            #print(f"Actual:{y:>8f} \n Pred: {pred:>8f} \n")
    test_loss /= size
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [21]:
image_train = ImageValuePair(img_train_paths,  transforms=pair_transforms)
train_dataloader = DataLoader(image_train, batch_size=8, shuffle=True)
image_test= ImageValuePair(img_test_paths, transforms=pair_transforms)
test_dataloader = DataLoader(image_test, batch_size=8, shuffle=True)

In [22]:
#train_images = next(iter(test_dataloader))

In [23]:
# #pick an image to view
# n=1

# train_images, train_ages= next(iter(test_dataloader))
# agesout = model(train_images.cuda())
# print(f"Feature batch shape: {train_images.size()}")
# img = train_images[n].squeeze()

# plt.imshow(img, cmap="gray")
# plt.show()
# print(agesout[n])

In [30]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01, amsgrad=True) # 0.0001
epochs = 150
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.568094  [    0/ 1080]
loss: 4.266264  [   80/ 1080]
loss: 2.507753  [  160/ 1080]
loss: 2.364590  [  240/ 1080]
loss: 3.404545  [  320/ 1080]
loss: 1.119358  [  400/ 1080]
loss: 2.188883  [  480/ 1080]
loss: 1.862412  [  560/ 1080]
loss: 2.674530  [  640/ 1080]
loss: 5.444207  [  720/ 1080]
loss: 2.901593  [  800/ 1080]
loss: 2.611867  [  880/ 1080]
loss: 1.559303  [  960/ 1080]
loss: 1.887712  [ 1040/ 1080]


d:\Anaconda\envs\fastai2\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Test Error: 
 Avg loss: 0.891849 

Epoch 2
-------------------------------
loss: 3.081317  [    0/ 1080]
loss: 1.434530  [   80/ 1080]
loss: 1.508414  [  160/ 1080]
loss: 1.768135  [  240/ 1080]
loss: 1.831609  [  320/ 1080]
loss: 1.581976  [  400/ 1080]
loss: 1.378571  [  480/ 1080]
loss: 1.478753  [  560/ 1080]
loss: 1.406853  [  640/ 1080]
loss: 2.210195  [  720/ 1080]
loss: 1.852059  [  800/ 1080]
loss: 1.983029  [  880/ 1080]
loss: 2.809550  [  960/ 1080]
loss: 1.766639  [ 1040/ 1080]
Test Error: 
 Avg loss: 0.900014 

Epoch 3
-------------------------------
loss: 1.933715  [    0/ 1080]
loss: 1.260950  [   80/ 1080]
loss: 1.563570  [  160/ 1080]
loss: 1.266353  [  240/ 1080]
loss: 2.019856  [  320/ 1080]
loss: 1.900375  [  400/ 1080]
loss: 1.671173  [  480/ 1080]
loss: 2.173006  [  560/ 1080]
loss: 2.365309  [  640/ 1080]
loss: 1.392095  [  720/ 1080]
loss: 1.980165  [  800/ 1080]
loss: 1.740018  [  880/ 1080]
loss: 0.771976  [  960/ 1080]
loss: 2.066819  [ 1040/ 1080]
Test Error

In [35]:
#pick an image to view
n=4

train_images, train_ages= next(iter(test_dataloader))
agesout = model(train_images.cuda())
print(f"Feature batch shape: {train_images.size()}")
img = train_images[n].squeeze()

plt.imshow(img, cmap="gray")
plt.show()
print(agesout[n])

RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 24.00 GiB total capacity; 20.52 GiB already allocated; 0 bytes free; 20.91 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
currentSave = "FirstTestAgePredict_manual_unet.pth"

In [26]:

torch.save(model, currentSave)

# model_scripted = torch.jit.script(model) # Export to TorchScript
# model_scripted.save(currentSave) # Save

In [27]:
model2 = torch.load(currentSave)


In [28]:
out = model2(img_test_paths[2])

TypeError: conv2d() received an invalid combination of arguments - got (str, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
